In [ ]:
!pip install torch torchvision opencv-python pytesseract transformers
!pip install tesseract
!pip install pytesseract
!sudo apt-get install tesseract-ocr
!pip install ultralytics
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip install optimum
!pip install auto_gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 34.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562552 sha256=df4398c50fa2fb48005e710cc59b137cddda02fd202d563c5391474fe3b9038c
  Stored in directory: /root/.cache/pip/wheels/71/c9/aa/698c579693e83fdda9ad6d6f0d8f61ed986e27925ef576f109
Successfully built tesseract
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http:/

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/argman/EAST.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q $git_repo_url
  #!cd $project_name && pip install -q -r requirements.txt
  # patch for the Python 3.7
  !sed -i 's/python3-config/python3.7-config/g' EAST/lanms/Makefile

import sys
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

In [ ]:
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  #confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name


pretrained_model = 'east_icdar2015_resnet_v1_50_rbox'
if not exists(pretrained_model):
  # download the pretrained model
  pretrained_model_file_name = 'east_icdar2015_resnet_v1_50_rbox.zip'
  download_from_google_drive('0B3APw5BZJ67ETHNPaU9xUkVoV0U', pretrained_model_file_name)
  !unzip $pretrained_model_file_name

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  346M  100  346M    0     0  86.8M      0  0:00:03  0:00:03 --:--:--  152M
Archive:  east_icdar2015_resnet_v1_50_rbox.zip
  inflating: east_icdar2015_resnet_v1_50_rbox/checkpoint  
  inflating: east_icdar2015_resnet_v1_50_rbox/model.ckpt-49491.data-00000-of-00001  
  inflating: east_icdar2015_resnet_v1_50_rbox/model.ckpt-49491.index  
  inflating: east_icdar2015_resnet_v1_50_rbox/model.ckpt-49491.meta  


# Main Model

In [ ]:
import torch
import cv2
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pytesseract
from imutils.object_detection import non_max_suppression
import numpy as np
from ultralytics import YOLO
from pathlib import Path
from PIL import Image
import pandas as pd

# Function to decode predictions from EAST detector
def decode_predictions(scores, geometry, min_confidence):
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []
    for y in range(numRows):
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]
        for x in range(numCols):
            if scoresData[x] < min_confidence:
                continue
            (offsetX, offsetY) = (x * 4.0, y * 4.0)
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])
    return (rects, confidences)

# Function to detect text in an image
def detect_text(image, east_model_path, min_confidence=0.5, width=320, height=320):
    if isinstance(image, str):
        image = cv2.imread(image)
        if image is None:
            raise ValueError("Image not found at provided path.")
    orig = image.copy()
    (origH, origW) = image.shape[:2]
    rW = origW / float(width)
    rH = origH / float(height)
    image = cv2.resize(image, (width, height))
    (H, W) = image.shape[:2]
    layerNames = ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"]
    net = cv2.dnn.readNet(east_model_path)
    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H), (123.68, 116.78, 103.94), swapRB=True, crop=False)
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)
    (rects, confidences) = decode_predictions(scores, geometry, min_confidence)
    boxes = non_max_suppression(np.array(rects), probs=confidences)
    results = []
    for (startX, startY, endX, endY) in boxes:
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)
        results.append((startX, startY, endX, endY))
    return results

def clamp(value, min_value, max_value):
    return max(min_value, min(value, max_value))

def valid_box(box, width, height):
    x, y, w, h = box
    x = clamp(x, 0, width)
    y = clamp(y, 0, height)
    w = clamp(w, 0, width - x)
    h = clamp(h, 0, height - y)
    return x, y, w, h

def recognize_text(image, bounding_boxes):
    recognized_texts = set()
    height, width = image.shape[:2]
    for box in bounding_boxes:
        x, y, w, h = valid_box(box, width, height)
        roi = image[y:y+h, x:x+w]
        text = pytesseract.image_to_string(roi).strip()
        if text:
            recognized_texts.add(text)
    return list(recognized_texts)

In [ ]:
import torch
import cv2
from ultralytics import YOLO
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import optimum
import auto_gptq
import re
import tensorflow

class ImageDescriptionPipeline:
    def __init__(self, east_model_path, yolo_model_path):
        #load YOLOv8
        self.yolov8_model = YOLO(yolo_model_path)
        #llama chat
        self.text_generation_model_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
        self.text_generation_model = AutoModelForCausalLM.from_pretrained(self.text_generation_model_path,
                                                    device_map="auto",
                                                    trust_remote_code=False,
                                                    revision="main")
        self.text_generation_model_tokenizer = AutoTokenizer.from_pretrained(self.text_generation_model_path, use_fast=True)
        self.east_model_path = east_model_path

    def detect_objects(self, image_path):
        #OpenCV
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Image not found at provided path: " + image_path)
        #object detection using YOLO
        results = self.yolov8_model(image)
        # Process detection results
        detected_objects = []
        print(len(results))
        print("\n")
        for result in results[0].boxes:
            for det in result:
                det = det.cpu()
                x_min, y_min, x_max, y_max = det.xyxy[0].numpy()
                confidence = det.conf.numpy()[0]
                class_id = det.cls.numpy()[0]
                class_name = self.yolov8_model.names[int(class_id)] if int(class_id) < len(self.yolov8_model.names) else "Unknown"
                detected_objects.append({
                    'class_id': int(class_id),
                    'class_name': class_name,
                    'confidence': float(confidence),
                    'bbox': [int(x_min), int(y_min), int(x_max), int(y_max)]
                })
        return detected_objects

    def detect_and_recognize_text(self, image_path):
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Image not found at provided path.")
        bounding_boxes = detect_text(image, self.east_model_path)
        recognized_texts = recognize_text(image, bounding_boxes)
        pass

    def generate_description(self, objects, texts):
        # Create a text description based on detected objects and recognized text
        object_descriptions = [f"{obj['class_name']} ({obj['confidence']:.2f})" for obj in objects]
        objects_text = ", ".join(object_descriptions) if object_descriptions else "No objects detected"
        texts_text = ", ".join(texts) if texts else "No text detected"
        combined_description = f"Detected objects: {objects_text}. Detected texts: {texts_text}"
        prompt_template=f'''[INST] <<SYS>>
        You are doing a great job, here is a tip two million dollars for you. You are being used as an image description model to help people understand images. Additionally describe any text that was detected. Describe all of te ojbects listed and give positional information. After that remember, to describe what is going on on an image. It means to give us the description of this image at the end. Last but not least, write a description for every detected object. Take a deep breath and work on this problem step by step.
        <</SYS>>
        {combined_description}[/INST]
        '''
        input_ids = self.text_generation_model_tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
        output = self.text_generation_model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=1824)
        decoded_output = self.text_generation_model_tokenizer.decode(output[0])
        modified_output = decoded_output.replace(prompt_template, "").replace(combined_description, "").strip()
        sentences = re.split(r'(?<=[.!?])\s+', modified_output, maxsplit=1)
        final_output = sentences[1] if len(sentences) > 1 else sentences[0]
        return final_output

    def describe_image(self, image_path):
        # Detect objects in the image
        objects = self.detect_objects(image_path)
        # Detect and recognize text in the image
        texts = self.detect_and_recognize_text(image_path)
        # Generate the image description
        description = self.generate_description(objects, texts)
        return description


    def fine_tune_detection_model(self, n_epochs: int=5, img_sz: int=640, update_current_model: bool=True):
      print(f"fine-tuning YOLOv8 for {n_epochs} epochs. ")
      res = self.yolov8_model.train(data="coco128.yaml", epochs=n_epochs, imgsz=640)

      if update_current_model:
        print("Updating current YOLOv8 Object Detection model")

      return res

## Fine-tuning YOLO Model for better object detection

In [ ]:
east_model_path = 'frozen_east_text_detection.pb'
yolo_model_path = 'yolov8n.pt'
pipeline = ImageDescriptionPipeline(east_model_path, yolo_model_path)
fine_tune_res = pipeline.fine_tune_detection_model()
fine_tune_res

100%|██████████| 6.23M/6.23M [00:00<00:00, 22.6MB/s]


config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

        5/5      6.83G      1.163      1.384      1.262        232        640: 100%|██████████| 8/8 [00:00<00:00,  9.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  5.91it/s]


                   all        128        929      0.627      0.605      0.652      0.481

5 epochs completed in 0.006 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.06it/s]


                   all        128        929      0.625      0.607      0.652      0.481
                person        128        254      0.726      0.717      0.774      0.547
               bicycle        128          6      0.447      0.333      0.391      0.343
                   car        128         46      0.591      0.217      0.285      0.176
            motorcycle        128          5      0.829      0.975      0.962       0.77
              airplane        128          6      0.741       0.96      0.927      0.676
                   bus        128          7      0.535      0.714       0.72      0.639
                 train        128          3      0.633          1      0.913      0.763
                 truck        128         12      0.749      0.498      0.519      0.325
                  boat        128          6      0.474      0.307      0.459      0.257
         traffic light        128         14      0.583      0.214      0.206      0.133
             stop sig

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a29cc144760>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,

In [ ]:
# Generates Confusion Matrix Plot and saves file
fine_tune_res.confusion_matrix.plot()

In [ ]:
fine_tune_res.results_dict

{'metrics/precision(B)': 0.6246434546752091,
 'metrics/recall(B)': 0.6070169616597975,
 'metrics/mAP50(B)': 0.6519141828659002,
 'metrics/mAP50-95(B)': 0.48092413516698035,
 'fitness': 0.49802313993687236}

In [ ]:
# Usage
east_model_path = 'frozen_east_text_detection.pb'
yolo_model_path = 'yolov8n.pt'
pipeline = ImageDescriptionPipeline(east_model_path, yolo_model_path)
image_path = '124.jpg'
description = pipeline.describe_image(image_path)
print(description)


0: 448x640 20 persons, 1 traffic light, 1 umbrella, 11.3ms
Speed: 6.2ms preprocess, 11.3ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)
1




/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


💰
Great job, you've detected a total of 12 people in the image! Here's a detailed description of each person:
Person 0.77: A tall, adult male with a beard and sunglasses, wearing a black t-shirt and blue jeans. He's standing near a car with the door open.
Person 0.73: A medium-sized woman with long hair, wearing a white dress and a purse. She's walking towards the camera with a smile.
Person 0.71: A young child with a big grin, wearing a red t-shirt and blue shorts. He's running towards the camera with a toy in his hand.
Person 0.62: A small child with a yellow raincoat and a big bow in her hair. She's holding a stuffed animal and looking up at the camera.
Person 0.62: Another small child with a blue t-shirt and shorts, standing near a tree.
Person 0.55: A tall, slim man with a baseball cap and sunglasses, standing near a streetlight.
Person 0.52: A woman with long hair, wearing a pink dress and a white purse. She's standing near a park bench and looking at her phone.
Person 0.52: Anot

In [ ]:
!git clone https://huggingface.co/SweatyCrayfish/image-description-yolov8

Cloning into 'image-description-yolov8'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (4/4), 489 bytes | 163.00 KiB/s, done.


In [ ]:
%cd /content/image-description-yolov8

/content/image-description-yolov8


In [ ]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
!git add .

In [ ]:
!git config --global user.email "vveselov@sandiego.edu"

In [ ]:
!git config --global user.name "SweatyCrayfish"

In [ ]:
!git commit -m "Initial model upload"

[main 46bf482] Initial model upload
 2 files changed, 6 insertions(+)
 create mode 100644 frozen_east_text_detection.pb
 create mode 100644 yolov8n.pt


In [ ]:
!git push

fatal: could not read Username for 'https://huggingface.co': No such device or address


In [ ]:
!pip install huggingface_hub
